### Import necessary libraries and modules

In [66]:
import os
import pickle
import time
import json
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional, Callable, Union
from functools import reduce
from io import BytesIO
from logging import WARNING

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torchvision import transforms, datasets

import flwr as fl
from flwr.common import (Metrics, EvaluateIns, EvaluateRes, FitIns, FitRes, MetricsAggregationFn, Scalar, logger, Parameters, NDArray, NDArrays)
from flwr.server.client_proxy import ClientProxy
from flwr.server.client_manager import ClientManager
from flwr.server.strategy.aggregate import weighted_loss_avg
import tenseal as ts

In [67]:
def create_encryption_context():
    context_params = {
        "scheme": ts.SCHEME_TYPE.CKKS,
        "poly_modulus_degree": 8192,
        "coeff_modulus_bits": [60, 40, 40, 60],
        "global_scale": 1 << 40  # Equivalent to 2 ** 40
    }
    
    # Initializing encryption context 
    enc_context = ts.context(**context_params)
    enc_context.generate_galois_keys()
    
    return enc_context


In [ ]:
class NeuralLayer:

    def __init__(self, identifier, weights):
        self.identifier = identifier  
        self.weights = weights 
  
    def name(self):
        return self.identifier
    
    def data(self):
        return self.weights

    def _operate(self, other, op):
        operand = other.data() if isinstance(other, NeuralLayer) else other
        return NeuralLayer(self.identifier, op(self.weights, operand))

    def add(self, other):
        return self._operate(other, np.add)
    
    def subtract(self, other):
        return self._operate(other, np.subtract)
    
    def multiply(self, other):
        return self._operate(other, np.multiply)

    def divide(self, other):
        return self._operate(other, lambda x, y: x / y)
    
    # Utility functions 
    def shape_info(self):
        return self.weights.shape

    def total_elements(self):
        return int(np.prod(self.weights.shape))

    def aggregate(self, mode="sum", axis=0):
        if mode == "sum":
            return NeuralLayer(f"{self.identifier}_sum", self.weights.sum(axis=axis))
        elif mode == "mean":
            return NeuralLayer(f"{self.identifier}_mean", np.mean(self.weights, axis=axis))
    
    # Decryption and Serialization Methods
    def decrypt_content(self, secret_key=None):
        return self.weights.tolist() if not secret_key else secret_key.decrypt(self.weights)
    
    def export(self):
        return {self.identifier: self.weights}


In [ ]:
class EncryptedLayer(NeuralLayer):
    def __init__(self, layer_id, weights, encryption_context=None):
        super().__init__(layer_id, weights)
        if isinstance(weights, (ts.tensors.CKKSTensor, bytes)):
            self.weights = weights
        else:
            self.weights = self._encrypt_weights(weights, encryption_context)
    
    def _encrypt_weights(self, raw_weights, context):
        return ts.ckks_tensor(context, raw_weights.cpu().numpy()) if context else raw_weights

    def _decrypt_weights(self, secret_key=None):
        return (self.weights.decrypt(secret_key) if secret_key else self.weights.decrypt()).tolist()

    def operate_layer(self, other, operation):
        other_weights = other.data() if isinstance(other, EncryptedLayer) else other
        try:
            new_weights = operation(self.weights, other_weights)
        except TypeError:
            print("Operation not supported for encrypted data format.")
            new_weights = []
        return EncryptedLayer(self.identifier, new_weights)

    def add_layer(self, other):
        return self.operate_layer(other, np.add)
    
    def subtract_layer(self, other):
        return self.operate_layer(other, np.subtract)
    
    def multiply_layer(self, other):
        return self.operate_layer(other, np.multiply)
    
    def divide_layer(self, other):
        try:
            return self.operate_layer(other, lambda x, y: x / y)
        except ZeroDivisionError:
            print("Warning: Division by zero encountered in encrypted data.")
            return EncryptedLayer(self.identifier, [])

    def layer_shape(self):
        return self.weights.shape if isinstance(self.weights, ts.tensors.CKKSTensor) else None

    def aggregate_layer(self, method="sum", axis=0):
        if method == "sum":
            result = self.weights.sum(axis=axis)
        elif method == "mean":
            result = self.weights.sum(axis=axis) / self.weights.shape[axis]
        else:
            raise ValueError("Invalid aggregation method.")
        
        return EncryptedLayer(f"{self.identifier}_{method}", result)
    
    def prepare_for_export(self):
        return {self.identifier: self.weights.serialize() if isinstance(self.weights, ts.tensors.CKKSTensor) else self.weights}


In [ ]:
def encrypt_weights(client_weights, encryption_context):
    encrypted_layers = []
    for layer_name, weights in client_weights.items():
        start = time.time()
        encrypted_layer = EncryptedLayer(layer_name, weights, encryption_context) if layer_name == 'fc3.weight' else NeuralLayer(layer_name, weights)
        encrypted_layers.append(encrypted_layer)
        print(layer_name, (time.time() - start))
    return encrypted_layers


def load_query(file_location):
    if not os.path.isfile(file_location):
        print("File not found.")
        return None, None
    with open(file_location, 'rb') as f:
        query_content = pickle.load(f)
    enc_context = query_content.pop("contexte")
    return query_content, enc_context


def save_query(file_location, query_data):
    with open(file_location, 'wb') as f:
        pickle.dump(query_data, f)


def decode_layer(layer_name, weights, context):
    if isinstance(weights, bytes):
        return EncryptedLayer(layer_name, ts.ckks_tensor_from(context, weights), context)
    elif isinstance(weights, ts.tensors.CKKSTensor):
        return EncryptedLayer(layer_name, weights, context)
    return NeuralLayer(layer_name, weights)


def restore_model(query_data, context):
    return [decode_layer(layer_name, weights, context) for layer_name, weights in query_data.items()]


In [68]:
def aggregate_weights(results):
    total_samples = sum([samples for _, samples in results])
    weighted_updates = [[layer * samples for layer in layers] for layers, samples in results]
    
    final_weights = [
        reduce(np.add, updates) / total_samples for updates in zip(*weighted_updates)
    ]
    return final_weights


def setup_tfrecord_files(output_path, record_count=10, prefix="train"):
    return [tf.io.TFRecordWriter(f"{output_path}/{prefix}{i}.tfrecord") for i in range(record_count)]


def build_vocab_table(vocab_dict, oov_buckets):
    vocab_keys = list(vocab_dict.keys())
    vocab_values = [tf.constant(vocab_dict[key], dtype=tf.int64) for key in vocab_keys]
    initializer = tf.lookup.KeyValueTensorInitializer(keys=vocab_keys, values=vocab_values)
    return tf.lookup.StaticVocabularyTable(initializer, num_oov_buckets=oov_buckets)


def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def load_vocab(filepath):
    with open(filepath, "r") as file:
        return json.load(file)


def serialize_example(mnemonics, label, encoding_type="mnemonic"):
    features = {}
    
    if encoding_type == "mnemonic":
        features["opcodes"] = bytes_feature(mnemonics.encode('UTF-8'))
    elif encoding_type == "id":
        features["opcodes"] = bytes_feature(np.array(mnemonics).tobytes())
    elif encoding_type == "api":
        features["APIs"] = bytes_feature(mnemonics.tobytes())
    elif encoding_type == "bytes":
        features["bytes"] = bytes_feature(mnemonics.encode('UTF-8'))

    features["label"] = int64_feature(label)
    example_proto = tf.train.Example(features=tf.train.Features(feature=features))
    return example_proto.SerializeToString()


def load_params(params_filepath):
    with open(params_filepath, "r") as file:
        return json.load(file)


In [69]:
def evaluate_model(model, data_loader, loss_function, device):
    model.eval()
    total_loss, accuracy, predictions, true_labels, probabilities = 0, 0, [], [], []
    softmax_fn = nn.Softmax(dim=1)

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted_probs = softmax_fn(outputs)
            probabilities.extend(predicted_probs.cpu().numpy())

            loss = loss_function(outputs, labels)
            total_loss += loss.item()

            true_labels.extend(labels.cpu().numpy())
            pred_classes = np.argmax(outputs.cpu().numpy(), axis=1)
            predictions.extend(pred_classes)
            batch_accuracy = (pred_classes == labels.cpu().numpy()).mean()
            accuracy += batch_accuracy

    probabilities = np.array(probabilities)
    avg_loss = total_loss / len(data_loader)
    avg_accuracy = accuracy / len(data_loader) * 100
    return avg_loss, avg_accuracy, predictions, true_labels, probabilities


def single_training_step(model, data_loader, loss_function, optimizer, device):
    model.train()
    step_loss, step_accuracy = 0, 0

    for _, (inputs, targets) in enumerate(data_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        step_loss += loss.item()
        loss.backward()
        optimizer.step()

        predicted_classes = torch.argmax(torch.softmax(outputs, dim=1), dim=1)
        step_accuracy += (predicted_classes == targets).float().mean().item()

    avg_loss = step_loss / len(data_loader)
    avg_accuracy = step_accuracy / len(data_loader) * 100
    return avg_loss, avg_accuracy


def training_loop(model, train_loader, val_loader, optimizer, loss_function, epochs, device):
    history = {"train_loss": [], "train_accuracy": [], "val_loss": [], "val_accuracy": []}

    for epoch in tqdm(range(epochs), desc="Training Progress", colour="blue"):
        train_loss, train_accuracy = single_training_step(
            model, data_loader=train_loader, loss_function=loss_function, optimizer=optimizer, device=device
        )
        val_loss, val_accuracy, *_ = evaluate_model(
            model, data_loader=val_loader, loss_function=loss_function, device=device
        )

        print(
            f"Epoch [{epoch + 1}/{epochs}] | "
            f"Train Loss: {train_loss:.4f} | Train Acc: {train_accuracy:.2f}% | "
            f"Val Loss: {val_loss:.4f} | Val Acc: {val_accuracy:.2f}%"
        )

        history["train_loss"].append(train_loss)
        history["train_accuracy"].append(train_accuracy)
        history["val_loss"].append(val_loss)
        history["val_accuracy"].append(val_accuracy)

    return history


In [70]:
def convert_ndarrays_to_params(ndarrays):
    return custom_ndarray_to_params(ndarrays)


def convert_params_to_ndarrays(parameters, context):
    with open("secret.pkl", 'rb') as f:
        stored_context = pickle.load(f)
    client_context = ts.context_from(stored_context["contexte"])
    return custom_params_to_ndarray(parameters, client_context)


def serialize_ndarray(array):
    return custom_serialize_ndarray(array)


def deserialize_ndarray(serialized_tensor, context):
    byte_stream = BytesIO(serialized_tensor)
    deserialized_array = np.load(byte_stream, allow_pickle=False)
    return deserialized_array


def custom_serialize_ndarray(array):
    if isinstance(array, ts.tensors.CKKSTensor):
        return array.serialize()
    byte_buffer = BytesIO()
    np.save(byte_buffer, array.cpu().detach().numpy() if isinstance(array, torch.Tensor) else array, allow_pickle=False)
    return byte_buffer.getvalue()


def custom_params_to_ndarray(parameters, context):
    return [deserialize_ndarray(tensor, context) for tensor in parameters.tensors]



In [71]:

def initialize_key_contexts(client_path="client_key.pkl", server_path="server_key.pkl"):
    enc_context = create_encryption_context()
    save_query(client_path, {"contexte": enc_context.serialize(save_secret_key=True)})
    save_query(server_path, {"contexte": enc_context.serialize()})

    _, client_enc_context = load_query(client_path)
    _, server_enc_context = load_query(server_path)

    client_context_obj = ts.context_from(client_enc_context)
    server_context_obj = ts.context_from(server_enc_context)

    print("Client context privacy:", "Yes" if client_context_obj.is_private() else "No")
    print("Server context privacy:", "Yes" if server_context_obj.is_private() else "No")



secret_path = "secret.pkl"
public_path = "server_key.pkl"
if os.path.exists(secret_path):
    print("it exists")
    _, context_client = read_query(secret_path)

else:
    initialize_key_contexts(client_path=secret_path, server_path=public_path)

n_qubits = 4
n_layers = 4
weight_shapes = {"weights": (n_layers, n_qubits)}

it exists


In [73]:
class FederatedClient(fl.client.NumPyClient):
    def __init__(self, client_id, model, train_loader, val_loader, device, config):
        self.client_id = client_id
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.batch_size = config['batch_size']
        self.save_results = config['save_results']
        self.paths = {
            "matrix": config['matrix_path'],
            "roc": config['roc_path'],
            "yaml": config['yaml_path']
        }
        self.context = config['context'] if config['he_enabled'] else None
        self.classes = config['classes']

    def get_parameters(self, _config):
        print(f"Fetching parameters for client {self.client_id}")
        return extract_model_parameters(self.model, self.context)

    def fit(self, parameters, config):
        print(f"Client {self.client_id}: Training with configuration: {config}")
        update_model_parameters(self.model, parameters, self.context)

        results = train_model(
            model=self.model,
            train_loader=self.train_loader,
            val_loader=self.val_loader,
            epochs=config['local_epochs'],
            learning_rate=float(config['learning_rate']),
            device=self.device
        )

        if self.save_results:
            save_training_metrics(self.save_results, config['local_epochs'], results, suffix=f"_Client_{self.client_id}")

        return extract_model_parameters(self.model, self.context), len(self.train_loader), {}

    def evaluate(self, parameters, config):
        print(f"Client {self.client_id} evaluating with configuration: {config}")
        update_model_parameters(self.model, parameters, self.context)

        loss, accuracy, y_pred, y_true, y_proba = evaluate_model(
            model=self.model, data_loader=self.val_loader, loss_function=nn.CrossEntropyLoss(), device=self.device
        )

        if self.save_results:
            save_evaluation_results(y_true, y_pred, y_proba, self.save_results, self.paths, self.classes)

        return float(loss), len(self.val_loader), {"accuracy": float(accuracy)}


In [ ]:
def extract_model_parameters(model, context=None):
    model_state = model.state_dict()
    if context:
        return [encrypt_layer_weights(name, param, context) for name, param in model_state.items()]
    return [param.cpu().numpy() for param in model_state.values()]

def update_model_parameters(model, parameters, context=None):
    state_dict = dict(zip(model.state_dict().keys(), parameters))
    if context:
        secret_key = context.secret_key()
        decrypted_state = {name: torch.Tensor(decrypt_layer(param, context, secret_key)) for name, param in state_dict.items()}
    else:
        decrypted_state = {name: torch.Tensor(param) for name, param in state_dict.items()}
    model.load_state_dict(decrypted_state, strict=True)
    print("Model parameters updated.")


In [75]:
def compute_weighted_average(metrics_list):
    total_samples = sum([sample_count for sample_count, _ in metrics_list])
    weighted_accuracy = sum([sample_count * metrics["accuracy"] for sample_count, metrics in metrics_list]) / total_samples
    return {"accuracy": weighted_accuracy}

def configure_evaluation(server_round, parameters, config):
    update_model_parameters(global_model, parameters)
    eval_loss, eval_accuracy, *_ = evaluate_model(global_model, test_loader, nn.CrossEntropyLoss(), DEVICE)
    print(f"Server round {server_round} evaluation loss: {eval_loss}, accuracy: {eval_accuracy}")
    return eval_loss, {"accuracy": eval_accuracy}


In [76]:
class CustomFederatedStrategy(fl.server.strategy.Strategy):
    def __init__(self, config):
        self.config = config
        super().__init__(
            fraction_fit=config['frac_fit'],
            fraction_evaluate=config['frac_eval'],
            min_fit_clients=config['min_fit_clients'],
            min_evaluate_clients=config['min_evaluate_clients'],
            min_available_clients=config['min_available_clients'],
            evaluate_fn=configure_evaluation if not config['he_enabled'] else None,
            on_fit_config_fn=lambda round_num: configure_training(round_num, config),
            initial_parameters=convert_to_parameters(get_initial_model_parameters())
        )

    def configure_fit(self, server_round, parameters, client_manager):
        sample_size, min_clients = self.sample_fit_clients(client_manager.num_available())
        clients = client_manager.sample(num_clients=sample_size, min_num_clients=min_clients)

        configurations = [create_fit_ins(client, parameters, server_round) for client in clients]
        return configurations

    def aggregate_fit(self, server_round, results, failures):
        if not results or (not self.config['accept_failures'] and failures):
            return None, {}
        aggregated_params = aggregate_client_updates(results)
        save_checkpoint(server_round, aggregated_params)
        return aggregated_params, {}

    def configure_evaluate(self, server_round, parameters, client_manager):
        if self.config['frac_eval'] == 0:
            return []
        sample_size, min_clients = self.sample_evaluate_clients(client_manager.num_available())
        clients = client_manager.sample(num_clients=sample_size, min_num_clients=min_clients)
        eval_ins = EvaluateIns(parameters, {})
        return [(client, eval_ins) for client in clients]


In [ ]:
def get_initial_model_parameters():
    return [param.cpu().numpy() for param in global_model.state_dict().values()]

def convert_to_parameters(ndarrays):
    return fl.common.Parameters(tensors=[array.tobytes() for array in ndarrays], tensor_type="numpy.ndarray")

def aggregate_client_updates(results):
    num_samples_total = sum([num_samples for _, num_samples in results])
    aggregated_updates = [
        sum(client_update * num_samples for client_update, num_samples in results) / num_samples_total
        for client_update in zip(*[client_update for client_update, _ in results])
    ]
    return convert_to_parameters(aggregated_updates)

def save_checkpoint(server_round, parameters):
    checkpoint_path = f"checkpoint_round_{server_round}.pt"
    torch.save(parameters, checkpoint_path)
    print(f"Checkpoint saved: {checkpoint_path}")


In [77]:
def select_device(preferred_device):
    return "cuda:0" if torch.cuda.is_available() and preferred_device != "cpu" else "cpu"

def get_cifar_classes():
    return ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def clean_hidden_files(path):
    for file in os.listdir(path):
        if file == ".DS_Store":
            os.remove(os.path.join(path, file))


### Set up the federated learning strategy

In [78]:
config = {
    "encryption_enabled": False,
    "data_directory": "data/",
    "dataset_name": "cifar",
    "yaml_output": "./results/FL/results.yml",
    "random_seed": 0,
    "worker_count": 0,
    "max_training_epochs": 10,
    "batch_size": 128,
    "validation_split": 10,
    "device_type": "gpu",
    "client_count": 10,
    "output_directory": "results/FL/",
    "confusion_matrix_filename": "confusion_matrix.png",
    "roc_filename": "roc.png",
    "model_checkpoint": "cifar_fl.pt",
    "minimum_fit_clients": 10,
    "minimum_available_clients": 10,
    "minimum_eval_clients": 10,
    "rounds_count": 20,
    "fit_fraction": 1.0,
    "eval_fraction": 0.5,
    "learning_rate": 1e-3
}


In [80]:
DEVICE = torch.device(select_device(config["device_type"]))
CLASS_NAMES = get_cifar_classes()
global_model = QuantumNet(num_classes=len(CLASS_NAMES)).to(DEVICE)

train_loaders, test_loader = load_cifar10_data(
    batch_size=config["batch_size"],
    client_count=config["client_count"],
    seed=config["random_seed"],
    data_dir=config["data_directory"]
)

strategy = CustomFederatedStrategy({
    "frac_fit": config["fit_fraction"],
    "frac_eval": config["eval_fraction"],
    "min_fit_clients": config["minimum_fit_clients"],
    "min_eval_clients": config["minimum_eval_clients"],
    "min_available_clients": config["minimum_available_clients"],
    "accept_failures": True,
    "initial_parameters": convert_to_parameters(get_initial_model_parameters()),
    "evaluate_fn": configure_evaluation if not config["encryption_enabled"] else None,
    "on_fit_config_fn": lambda round_num: configure_training(round_num, config),
    "he_enabled": config["encryption_enabled"]
})




In [82]:
def create_client(client_id: str) -> FederatedClient:
    return configure_client(
        client_id=client_id,
        model_path=config["model_checkpoint"],
        save_paths={
            "results": config["output_directory"],
            "matrix": config["confusion_matrix_filename"],
            "roc": config["roc_filename"],
            "yaml": config["yaml_output"]
        },
        batch_size=config["batch_size"],
        train_loaders=train_loaders,
        val_loaders=train_loaders,  # Reusing train loaders as val loaders for this config
        device=DEVICE,
        classes=CLASS_NAMES,
        encryption=config["encryption_enabled"]
    )

configure_simulation(
    client_fn=create_client,
    client_count=config["client_count"],
    rounds=config["rounds_count"],
    strategy=strategy,
    device=DEVICE
)

cifar
Files already downloaded and verified
Files already downloaded and verified
The training set is created for the classes : ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [83]:

def load_encryption_context(path):
    if os.path.exists(path):
        with open(path, 'rb') as f:
            stored_context = pickle.load(f)
        return ts.context_from(stored_context["contexte"])
    return None

def decrypt_checkpoint(checkpoint, secret_key):
    server_query, server_context = load_query("server_key.pkl")
    server_context = ts.context_from(server_context)
    for layer_name in checkpoint["model_state_dict"]:
        checkpoint["model_state_dict"][layer_name] = torch.tensor(
            decode_layer(layer_name, server_query[layer_name], server_context).decrypt_content(secret_key)
        )

def configure_training(server_round, config):
    return {
        "learning_rate": str(config["learning_rate"]),
        "batch_size": str(config["batch_size"]),
        "server_round": server_round,
        "local_epochs": config["max_training_epochs"]
    }

def print_initial_config(config):
    print("\n===== Simulation Configuration =====")
    for key, value in config.items():
        print(f"{key}: {value}")
    print("====================================\n")

print_initial_config(config)
print("Flower Version:", fl.__version__)
print("PyTorch Version:", torch.__version__)
print("Training Device:", DEVICE)


INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout


flwr 1.12.0
numpy 1.26.4
torch 2.4.0
torchvision 0.19.0
Training on cuda:0
Start simulation


2024-10-29 04:10:54,844	INFO worker.py:1753 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'GPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.19.2.2': 1.0, 'object_store_memory': 8593143398.0, 'memory': 17186286798.0, 'accelerator_type:T4': 1.0, 'CPU': 4.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_gpus': 1, 'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Updated model


INFO :      initial parameters (loss, other metrics): 2.302999475334264, {'accuracy': 10.027689873417721}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


...

Server-side evaluation loss 2.302999475334264 / accuracy 10.027689873417721
Updated model


INFO :      fit progress: (20, 2.302999475334264, {'accuracy': 10.027689873417721}, 164.4677557269997)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)


Server-side evaluation loss 2.302999475334264 / accuracy 10.027689873417721


(TemporaryActor pid=2747) 
(TemporaryActor pid=2747) Exception raised in creation task: The actor died because of an error raised in its creation task, ray::ClientAppActor.__init__() (pid=2747, ip=172.19.2.2, actor_id=1a1ca73f26f08db3c18ccd1601000000, repr=<flwr.simulation.ray_transport.ray_actor.FunctionActorManager._create_fake_actor_class.<locals>.TemporaryActor object at 0x7da4c09e3df0>)
(TemporaryActor pid=2747) RuntimeError: The actor with name ClientAppActor failed to import on the worker. This may be because needed library dependencies are not installed in the worker environment:
(TemporaryActor pid=2747) ray::ClientAppActor.__init__() (pid=2747, ip=172.19.2.2, actor_id=1a1ca73f26f08db3c18ccd1601000000, repr=<flwr.simulation.ray_transport.ray_actor.FunctionActorManager._create_fake_actor_class.<locals>.TemporaryActor object at 0x7da4c09e3df0>)
(TemporaryActor pid=2747) ModuleNotFoundError: No module named 'flwr.client.client_app'
(TemporaryActor pid=2746) Exception raised in cr

Simulation Time = 178.6981828212738 seconds
